In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
os.chdir('/content/drive/My Drive/Carlos_Research_2')
!pwd

In [ ]:
!ls

# Imports

In [ ]:
!pip install sparse

In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   #if like me you do not have a lot of memory in your GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "" #then these two lines force keras to use your CPU
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, \
LeakyReLU, Conv2DTranspose, ReLU, Reshape, Concatenate, Input
from keras.utils import to_categorical
from tensorflow.image import psnr
from sklearn.preprocessing import OneHotEncoder
import random
import itertools
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sparse
from EMDataGenerator import EMDataGenerator
  # not scipy sparse because that is not how michael encoded it

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3519232288690082470
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 814380144862555734
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5175092192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13062917530469662014
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2116889883881068343
physical_device_desc: "device: XLA_GPU device"
]


# load data - small size

In [ ]:
!ls em_data_10a_2channels/

In [ ]:
with open(r"em_data_10a_2channels/X_list_10a_2channel.pkl", 'rb') as f:
  # despite the name, it has dimensions 84x54x98
  X_3d = pickle.load(f)

In [ ]:
with open(r"em_data_10a_2channels/y_list_10a_2channel.pkl", 'rb') as f:
  y_2d = pickle.load(f)

In [ ]:
print(len(X_3d))
print(type(X_3d))
print(X_3d[0].shape)

In [ ]:
print(len(y_2d))
print(type(y_2d))
print(y_2d[0].shape)

In [ ]:
# every element in the X_3d list is a sparse matrix, convert it to a 3D numpy array
# takes too large of a memory, only process the first 3 matrix
X_array = []
for i in range(3):
  X_array.append(sparse.COO.todense(X_3d[i]))

In [ ]:
X_array[i].shape

In [ ]:
# change X to be an array of input shape (#_samples, x, y, z)
# change y to be an array of output shape (#_samples, x, y)
# also crop them to be squares/cubes
# but for the 3D, do not take the canter on the z-dim, take the front because those are closer to the camera

X_train = np.zeros(shape=(3, 32, 32, 32, 2))
y_train = np.zeros(shape=(3,32,32,1))
  # must add a channel dimension even if we only have 1 channel
  # the keras model require the extra channel dimension to do 2d and 3d convolutions

for i in range(3):
  X_train[i, :, :, :, :] = X_array[i][26:58, 11:43, 0:32,:]

for i in range(3):
  y_train[i, :, :, 0]  = y_2d[i][11:43, 26:58]

In [ ]:
np.unique(X_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

### Keras DataLoader

In [3]:
X_filename = '../em_data/em_data_32x32_043021/X_list_32x32x32.pkl'
y_filename = '../em_data/em_data_32x32_043021/y_list_full_32x32.pkl'
defocus_filename = '../em_data/em_data_32x32_043021/defocus_list_32x32.pkl'

# This is a bit confusing, but train and validation are randomly selected from train_val_range so that 
# validation is just new defocus views of lattices we've seen
# Test is held out lattices, all of test_range.
train_val_range = [0.,0.9]
train_shuffle = [0.,0.9]
val_shuffle = [0.9,1.]
test_range = [0.9,1.]
train_generator = EMDataGenerator(X_filename,y_filename,train_val_range,train_shuffle,5,defocus_filename)
valid_generator = EMDataGenerator(X_filename,y_filename,train_val_range,val_shuffle,5,defocus_filename)
test_generator = EMDataGenerator(X_filename,y_filename,test_range,None,5,defocus_filename)

In [4]:
X,y = train_generator[22]
print(len(X))
print(X[0].shape)
print(X[1].shape)
print(y.shape)
print(y.min(),y.max())
print(len(train_generator))

2
(5, 64, 64, 64, 2)
(5, 9)
(5, 64, 64, 1)
0.0 1.2196909
1880


### Adding metrics
Track PSNR and MSE through training (not sure why val_loss and val_mse_metric are not equal, but loss and mse_metric are equal...validation loss is calculated differently for some reason).
These metrics are passed in to model.compile(metrics=[])

In [5]:
# Calculate training y pixel intensity range, for PSNR function
y_range = np.array(train_generator.y).max() - np.array(train_generator.y).min()

def psnr_metric(y_true,y_pred):
    return psnr(y_true,y_pred,y_range)

def mse_metric(y_true,y_pred):
    return keras.losses.MSE(y_true,y_pred)

# model setup

In [6]:
sample_shape = (64,64,64,2)

In [7]:
def conditional_model(sample_shape=(32,32,32,2), defocus_1hot_shape = (9,)):
  '''
  takes in 3D input of shape (dim,dim,dim,2) and outputs a grey-scale 2-D image of shape (dim,dim,1).
  dim should be an power of 2 integer
  sample_shape:
    the shape of 1 sample, should be (dim,dim,dim,2)
  defocus_1hot_shape_shape:
    the shape of 1 row of one-hot-endoed defocus parameter of the input sample, should be (#_unique_defocus-1, )
  '''

  dim = sample_shape[0]
  f = int(32/(dim/32))
    # the starting filter size

  iter = int(np.log2(dim)) - 1
    # takes one less iteration because we want the shape to stop at 2, not 1
    # 4 for 32, 5 for 64, 6 for 128, 7 for 256

  # Create the model
  input_voxel = Input(shape=sample_shape)
  defocus = Input(shape=defocus_1hot_shape, dtype='float32')
  
  # add first layer that takes in the input
  encoder = Conv3D(filters=f, 
                  kernel_size=(4, 4, 4), 
                  strides=(2, 2, 2),
                  padding='same', 
                      # `same` just means as long as even just the left most 1 column of your kernel is still in the sample matrix, you will use padding to fill the parts that ran over the matrix and finish that mapping
                      # if you keep moving till you kernel does not overlap with your matrix at all we will stop and won't pad anymore
                  use_bias=False,
                  input_shape=sample_shape)(input_voxel)
  # model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  encoder = BatchNormalization(center=True, scale=True)(encoder)
  encoder = LeakyReLU(alpha=0.2)(encoder)
  # now the output should have shape (dim/2, dim/2, dim/2, f)

  # add middel encoder layers
  for i in range(1, iter): # 1~iter-1
    encoder = Conv3D(filters=f*(2**i), 
                    kernel_size=(4, 4, 4), 
                    strides=(2, 2, 2),
                    padding='same',
                    use_bias=False)(encoder)
    encoder = BatchNormalization(center=True, scale=True)(encoder)
    encoder = LeakyReLU(alpha=0.2)(encoder)
  
  # now the output should have shape (2, 2, 2, f*(2**i))

  # add latent layer
  encoder = Conv3D(filters=100, 
                  kernel_size=(2, 2, 2), 
                  strides=(1, 1, 1),
                  padding='valid',
                  use_bias=False)(encoder)
  encoder = LeakyReLU(alpha=0.2)(encoder)
    # VALID : Don't apply any padding
    # now the output should have shape (1, 1, 1, 100)

  encoder = Reshape((1,1,100))(encoder)
    # must reshape from a 3-D structure with 100 channels to a 2-D image having 100 channels
    # so Conv2DTranspose can work properly
    # now the output should have shape (1, 1, 100)

  defocus_vector = Reshape((1,1,defocus_1hot_shape[0]))(defocus)
    # reshape defocus vector to have shape (1, 1, defocus_input_shape)
    # must have the same dimension shape to concatenate with the latent vector
  latent_vector = Concatenate()([encoder, defocus_vector])
    # now the output should have shape (1,1,100+defocus_1hot_shape) now

  # add first blow-up decoder layer
  decoder = Conv2DTranspose(filters=f*(2**i),
                            kernel_size=(2,2),
                            strides=(1,1),
                            padding='valid',
                            use_bias=False
                            )(latent_vector)
  decoder = BatchNormalization(center=True, scale=True)(decoder)
  decoder = ReLU()(decoder)
  # now the output should have shape (2, 2, f*(2**i))

  # add middle decoder layers
  for i in range(iter-2, -1, -1 ): #iter-2 ~ 0
    decoder = Conv2DTranspose(filters=f*(2**i),
                              kernel_size=(4,4),
                              strides=(2,2),
                              padding='same',
                              use_bias=False
                              )(decoder)
    decoder = BatchNormalization(center=True, scale=True)(decoder)
    decoder = ReLU()(decoder)
  
  # now the output should have shape (dim/2, dim/2, f)

  # add final deocder output layer
  decoder = Conv2DTranspose(filters=1,
                            kernel_size=(4,4),
                            strides=(2,2),
                            padding='same',
                            use_bias=False
                            )(decoder)
  img = Dense(1, activation='tanh')(decoder)
  # now the output should have shape (dim, dim, 1)

  cond_model = Model([input_voxel,defocus], img)

  return cond_model

In [8]:
cond_model = conditional_model(sample_shape)
cond_model.compile(loss='mean_squared_error',
                   optimizer=keras.optimizers.Adam(lr=0.001),
                   metrics=[psnr_metric,mse_metric])
print("input shape:", sample_shape)
cond_model.summary()

input shape: (64, 64, 64, 2)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 64, 2 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 32, 32, 1 2048        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32, 1 64          conv3d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 32, 1 0           batch_normalization_1[0][0]      
_______________________________________________________________

In [ ]:
# Fit with DataGenerators
history = cond_model.fit(train_generator,
          validation_data = valid_generator,
          epochs = 400)

Epoch 1/400
1405/1880 [=====================>........] - ETA: 17s - loss: 0.0086 - psnr_metric: 21.6130 - mse_metric: 0.0086

In [ ]:
# summarize history for loss
plt.figure(figsize=[8,8])

plt.plot(history.history['mse_metric'])
plt.plot(history.history['val_mse_metric'])
plt.title('Loss curve')
plt.ylabel('MSE loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
# summarize history for PSNR
plt.figure(figsize=[8,8])

plt.plot(history.history['psnr_metric'])
plt.plot(history.history['val_psnr_metric'])
plt.title('PSNR curve')
plt.ylabel('PSNR')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')


In [ ]:
print(np.max(history2['val_psnr_metric']))
print(np.max(history.history['val_psnr_metric']))

In [ ]:
model_dir = '../../models/model_050221/'
model_file = 'model_32x32_200epochs_050121.pth'

# Save model
cond_model.save(model_dir+model_file)

# Save history file

hist_file = "history.pkl"
with open(model_dir+'history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
model_dir = '../../models/model_050121/'
model_file = 'model_32x32_200epochs_050121.pth'
# Loading history
with open(model_dir+'history.pkl', 'rb') as f:
    history2 = pickle.load(f)
history2

### Plotting

In [ ]:
X,y = train_generator[80]
preds = cond_model.predict(X)
k = 5
plt.figure(figsize=[7,20])
for i in range(k):
    se = np.linalg.norm(y[i,:,:,0] - preds[i,:,:,0])
    plt.subplot(k,2,i*2+1)
    if i == 0:
        plt.title("Predicted")
    plt.imshow(preds[i,:,:,0],cmap='Greys')
    plt.ylabel("Defocus: {}nm".format(np.argmax(X[1][i])))
    plt.xlabel("Squared Error: {:2.6f}".format(se))
    plt.subplot(k,2,i*2+2)
    if i == 0:
        plt.title("Actual")
    plt.imshow(y[i,:,:,0],cmap='Greys')

In [ ]:
y.max()

### Load model

In [ ]:
model = keras.models.load_model('cond_model_100epochs_042821.pth')

In [ ]:
preds = model.predict(X)

In [ ]:
k=2
plt.subplot(1,2,1)
plt.imshow(preds[k,:,:,0],cmap='Greys')
plt.subplot(1,2,2)
plt.imshow(y[k],cmap='Greys')

In [ ]:
preds.max()

# load data - original size

can't do it right now, takes up too much memory and kills session


In [ ]:
with open(r"em_data/X_list_840x540x980.pkl", 'rb') as f:
  X_3d = pickle.load(f)

In [ ]:
with open(r"em_data/y_list_840x540.pkl", 'rb') as f:
  y_2d = pickle.load(f)

In [ ]:
print(len(X_3d))
print(type(X_3d))

In [ ]:
print(len(y_2d))
print(type(y_2d))

In [ ]:
# every element in the X_3d list is a sparse matrix, convert it to a 3D numpy array
# takes too large of a memory, only process the first 1 matrix
X_array = []
for i in range(1):
  X_array.append(sparse.COO.todense(X_3d[i]))

In [ ]:
# change X to be an array of input shape (#_samples, x, y, z)
# change y to be an array of output shape (#_samples, x, y)

X_train = np.zeros(shape=(1, 840, 540, 980))
X_train[0] = X_array[0]

In [ ]:
np.unique(X_train)

NameError: name 'y' is not defined